# Corona Health - Psychological Health of adults ('Parents') Study

## Purpose of this Notebook
- [ ] Clean the dataset and save cleaned version
- [x] Get an statistical overview
    - [x] How many users?
    - [x] How many assessments (= filled out questionnaires)?
    - [x] Date range of the dataset?
    - [x] User-assessment distribution
- [ ] Potential target for classification?
- [ ] Potential features for classification?
    
    

#### Statistical overview

In [2]:
# imports
import pandas as pd
from datetime import date, datetime

In [3]:
# to import own modules, a workaround - assuming the curent working directory is the folder (your/local/path/UsAs/src/d01_analysis
import sys
sys.path.insert(0, "../..")

from src.d00_utils import helpers

In [4]:
# read in dataframe
# assuming current working directory is the folder (your/local/path/UsAs/src/d01_analysis)
df = pd.read_csv('../../data/d01_raw/ch/22-10-05_rki_parent_followup.csv')
df_baseline = pd.read_csv('../../data/d01_raw/ch/22-10-05_rki_parent_baseline.csv')

In [5]:
# create result dict to save to disk
result = dict()

In [6]:
# Number of unique users
print('Number of user:\t', df.user_id.nunique())
result['n_users'] = df.user_id.nunique()

Number of user:	 1474


In [7]:
# number of unique answers
print('Number of assessments:\t', df.answer_id.nunique())
result['n_assessments'] = df.answer_id.nunique()

Number of assessments:	 11081


In [8]:
# date range
form = '%Y-%m-%d %H:%M:%S'
date_start = df.created_at.min()
result['First assessment from'] = date_start
date_start = datetime.strptime(date_start, form)
date_end = df.created_at.max()
result['Latest assessment from'] = date_end
date_end = datetime.strptime(date_end, form)



print('Start:\t', date_start)
print('End:\t', date_end)

delta = date_end.date()-date_start.date()

print('\nDate Range in')
print('Years:\t', delta.days/365)
print('Months:\t', delta.days/12)
print('Days:\t', delta.days)


result['Time range in days'] = delta.days

Start:	 2020-07-21 14:13:47
End:	 2022-09-30 16:30:08

Date Range in
Years:	 2.1945205479452055
Months:	 66.75
Days:	 801


#### Distribution of filled out questionnaires

In [9]:
bins = [0, 1, 2, 3, 5, 10, 100, 1000]
ser = pd.cut(df.user_id.value_counts(), bins = bins).value_counts().sort_index()
ser
result.update(dict(ser))

### Average period length between two filled out questionnaires

In [10]:
# res = result
res = helpers.find_schedule_pattern(df, form='%Y-%m-%d %H:%M:%S', date_col_name='created_at')
result.update(res)

### Age Distribution

The following table shows the number of users for each age.

In [11]:
df_baseline.groupby('alter')['user_id'].count()

alter
18     50
19     41
20     42
21     48
22     58
       ..
79      1
80      2
82      2
84      1
120     2
Name: user_id, Length: 66, dtype: int64

Now we compute the mean age and the standard deviation

In [12]:
avg_age = df_baseline['alter'].mean()
std_age = df_baseline['alter'].std()

result['avg_age'] = avg_age
result['std_age'] = std_age

print("Durschnittliches Alter: %4.2f Jahre" % avg_age)
print("Standardabweichung Alter: %4.2f Jahre" % std_age)

Durschnittliches Alter: 41.18 Jahre
Standardabweichung Alter: 13.92 Jahre


### Sex Distribution

We calculate the number of users that self identified their sex as male (0), female (1) or other (2)

In [21]:
df_sex = pd.DataFrame(df_baseline.groupby('geschlecht')['user_id'].count())
df_sex['label'] = ('male', 'female', 'other')

result['n_users_male'] = df_sex['user_id'][0.0]
result['n_users_female'] = df_sex['user_id'][1.0]
result['n_users_other'] = df_sex['user_id'][2.0]

print(df_sex)
print("\n{} users without submitted sex".format(df_baseline['user_id'].count() - (result['n_users_male'] + result['n_users_female'] + result['n_users_other'])))

            user_id   label
geschlecht                 
0              1436    male
1              1213  female
2                21   other

0 users without submitted sex


### Country Statistics

We calculate how many users participated by country.

In [17]:
country_series = df_baseline.groupby('region')['user_id'].count()
country_series

region
13        1
151       6
154      10
155      12
21        1
30        1
39        1
de     2638
Name: user_id, dtype: int64

We also calculate the percentage of german-based users in the dataset

In [20]:
result['n_users_german'] = country_series['de']
result['n_users_non_german'] = country_series.sum()
print('{:.2f}% german-based users in dataset'.format(country_series['de'] / country_series.sum() * 100))

98.80% german-based users in dataset


### Statistical overview

In [10]:
result

{'n_users': 1474,
 'n_assessments': 11081,
 'First assessment from': '2020-07-21 14:13:47',
 'Latest assessment from': '2022-09-30 16:30:08',
 'Time range in days': 801,
 Interval(0, 1, closed='right'): 449,
 Interval(1, 2, closed='right'): 228,
 Interval(2, 3, closed='right'): 137,
 Interval(3, 5, closed='right'): 185,
 Interval(5, 10, closed='right'): 210,
 Interval(10, 100, closed='right'): 261,
 Interval(100, 1000, closed='right'): 4,
 'avg hours between two assessments': 488.5354808402132,
 'avg days between two assessments': 20.355645035008884,
 'std_hours': 690.5613713522175,
 'std_days': 28.773390473009062}

In [11]:
# read in codebook and reduce to columns and rows of interest
cb = pd.read_excel('../../data/d00_helpers/codebook/ch/rki_parent.xlsx', sheet_name='FollowUp', header=4)
cb = cb[cb.elementtype=='question']
cb = cb.iloc[:, :30]

C:\Users\joa24jm\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\worksheet\_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)
